In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="openai/gpt-oss-120b")

In [4]:
from typing import TypedDict, Annotated, List
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    ToolMessage
)
from langgraph.graph import START, END, StateGraph


In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    approved: bool


In [6]:
def ai_proposal(state: AgentState):
    response = llm.invoke(state["messages"])
    return {
        "messages": [AIMessage(content=response.content)],
        "approved": False
    }

In [7]:
from langgraph.types import interrupt

def human_review(state: AgentState):
    decision = interrupt({
        "question": "Do you approve this answer? (yes / no)",
        "ai_output": state["messages"][-1].content
    })

    return {
        "messages": [HumanMessage(content=decision)],
        "approved": decision.lower() == "yes"
    }

In [8]:
def route_after_review(state: AgentState):
    if state["approved"]:
        return "end"
    else:
        return "ai_retry"


In [9]:
graph = StateGraph(AgentState)

graph.add_node("ai", ai_proposal)
graph.add_node("human", human_review)
graph.add_node("ai_retry", ai_proposal)

graph.add_edge(START, "ai")
graph.add_edge("ai", "human")

graph.add_conditional_edges(
    "human",
    route_after_review,
    {
        "end": END,
        "ai_retry": "ai"
    }
)

app = graph.compile()


In [10]:
result = app.invoke({
    "messages": [HumanMessage(content="Explain LangGraph in simple words")],
    "approved": False
})

In [11]:
result = app.invoke(
    {
        "messages": [HumanMessage(content="yes")],
        "approved": False
    },
    resume=True
)


In [12]:
result

{'messages': [HumanMessage(content='yes', additional_kwargs={}, response_metadata={}, id='1408053c-61d9-4c4e-b5c7-9d81ed162c3a'),
  AIMessage(content='Sure! How can I assist you today? If you have a question or need help with something, just let me know.', additional_kwargs={}, response_metadata={}, id='4728ede4-a6ff-44d8-8f7f-7a48f76361a5', tool_calls=[], invalid_tool_calls=[])],
 'approved': False,
 '__interrupt__': [Interrupt(value={'question': 'Do you approve this answer? (yes / no)', 'ai_output': 'Sure! How can I assist you today? If you have a question or need help with something, just let me know.'}, id='513269e6d6dbcb6bda60dfaf0b17e0ec')]}